In [1]:
import time
import mysql.connector
import pandas as pd
import os
import warnings

from datetime import datetime, timedelta
warnings.filterwarnings('ignore')
start_time = time.time()

In [2]:
# Função para conectar ao banco de dados
def get_db_connection():
    return mysql.connector.connect(
        host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
        database='DW_DESENV',
        user='admin',
        password='yb222KraY7PTN0jbH7P3'
    )

# Função para contar registros em uma tabela
def obter_qtd_registros(tabela, connection):
    cursor = connection.cursor()
    cursor.execute(f"SELECT COUNT(*) FROM {tabela}")
    qtd = cursor.fetchone()[0]
    cursor.close()
    return qtd

# Função para obter a última data útil antes de um determinado dia
def obter_ultima_data_util(data_atual, connection):
    query = f'''
    SELECT MAX(DtReferencia) FROM DW_CORPORATIVO.Dm_Calendario dc
    WHERE dc.DtReferencia < '{data_atual}' AND dc.Feriado = 0 AND dc.FimSemana = 0
    '''
    cursor = connection.cursor()
    cursor.execute(query)
    ultima_data_util = cursor.fetchone()[0]
    cursor.close()
    return ultima_data_util.strftime('%Y-%m-%d') if ultima_data_util else None

# Função para verificar registros ausentes
def verificar_registros_ausentes(connection):
    dt_hoje = datetime.today().strftime('%Y-%m-%d')
    dt_ultima_util = obter_ultima_data_util(dt_hoje, connection)
    
    if not dt_ultima_util:
        return 0, 'Nenhum', []

    query = f'''
    SELECT DISTINCT fp2.NmFundo
    FROM DW_CORPORATIVO.Ft_Patrimonio fp2
    WHERE fp2.DtPosicao = '{dt_ultima_util}'
    AND NOT EXISTS (
        SELECT 1 FROM DW_CORPORATIVO.Ft_Patrimonio fp
        WHERE fp.DtPosicao = '{dt_hoje}' AND fp.NmFundo = fp2.NmFundo
    )
    '''
    cursor = connection.cursor()
    cursor.execute(query)
    fundos_ausentes = [row[0] for row in cursor.fetchall()]
    cursor.close()

    qtd_ausentes = len(fundos_ausentes)
    detalhes_ausentes = ', '.join(fundos_ausentes) if qtd_ausentes > 0 else 'Nenhum'
    
    return qtd_ausentes, detalhes_ausentes, fundos_ausentes

# Função para inserir log no banco
def registrar_log(fonte, qtd_inseridos, qtd_ausentes, detalhes_ausentes, fundos_ausentes, duracao_execucao, connection):
    cursor = connection.cursor()
    
    # Inserir log principal na tabela ETL_Log
    log_sql = '''
    INSERT INTO ETL_Log (DtExecucao, FonteDados, QtdRegistrosInseridos, QtdRegistrosAusentes, DetalhesRegistrosAusentes, DuracaoExecucao)
    VALUES (NOW(), %s, %s, %s, %s, %s)
    '''
    cursor.execute(log_sql, (fonte, qtd_inseridos, qtd_ausentes, detalhes_ausentes, duracao_execucao))
    connection.commit()  # Confirma a inserção do log antes de prosseguir
    
    # Debug: Verificar se há fundos ausentes antes da inserção
    if qtd_ausentes > 0:
        print(f"⚠️ Fundos ausentes detectados para {fonte}: {fundos_ausentes}")
        
        # Inserir cada fundo ausente individualmente para evitar problemas
        fundos_sql = '''
        INSERT INTO ETL_FundosAusentes (DtExecucao, FonteDados, Fundo) VALUES (NOW(), %s, %s)
        '''
        for fundo in fundos_ausentes:
            try:
                cursor.execute(fundos_sql, (fonte, fundo))
                connection.commit()  # Confirma cada inserção imediatamente
                print(f"✅ Fundo inserido: {fundo}")
            except Exception as e:
                print(f"❌ Erro ao inserir {fundo}: {e}")

    cursor.close()

# Singulare

In [3]:
# Caminho do arquivo Excel
caminho_arquivo_excel = '//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Patrimonio/FundCotationEvolutionMap.xls'

# Lê todas as planilhas do arquivo Excel
todos_sheets = pd.read_excel(caminho_arquivo_excel, sheet_name=None)

# Dicionário para armazenar os DataFrames processados
dfs_processados = {}

# Itera sobre cada sheet
for nome_do_sheet, agr_sheet in todos_sheets.items():
    # Obtém o nome do fundo na célula específica
    NmFundo1 = agr_sheet['Unnamed: 0'][3].replace('Cliente: ', '')

    # Exclui as 5 primeiras linhas e a última
    agr_sheet = agr_sheet.iloc[5:]
    agr_sheet = agr_sheet.iloc[:-1]

    # Cria uma lista com os nomes das colunas da primeira linha de valores
    colunas = []
    for i in range(len(agr_sheet.columns)):
        x = agr_sheet.iloc[0][i]
        colunas.append(x)
    
    # Renomeia as colunas
    agr_sheet.columns = colunas

    # Exclui a primeira linha (já usada para as colunas)
    agr_sheet = agr_sheet.iloc[1:]

    # Adiciona a coluna "CARTEIRA" com o nome do fundo
    agr_sheet['CARTEIRA'] = NmFundo1

    # Armazena o DataFrame processado no dicionário
    dfs_processados[nome_do_sheet] = agr_sheet


## seleciona os sheets existentes
arquivo_excel = pd.ExcelFile(caminho_arquivo_excel)
len(arquivo_excel.sheet_names)

FileNotFoundError: [Errno 2] No such file or directory: '//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Patrimonio/FundCotationEvolutionMap.xls'

In [4]:
stop = 0
df_singulare = pd.DataFrame()

for i in range(len(arquivo_excel.sheet_names)):
    
    agr_sheet1 = dfs_processados[f'Sheet{i+1}']
    
    df_singulare = pd.concat([df_singulare,agr_sheet1], ignore_index=True)
    
print(arquivo_excel.sheet_names)

NameError: name 'arquivo_excel' is not defined

# Gera dataframe

In [5]:
df_final = df_singulare #pd.concat([agr_sheet1, agr_sheet2, agr_sheet3, agr_sheet4, agr_sheet5, agr_sheet6,agr_sheet7], axis=0, ignore_index=True)
df_final['CARTEIRA'] = df_final['CARTEIRA'].str.strip() # remove espaços

# df_final = agr_sheet1
df_final['DocFundo'] = df_final['CARTEIRA']
df_final['NmGestor'] = 'CATALISE INVEST LTDA'
df_final['NmAdministrador'] = 'SINGULARE CORRETORA DE TITULOS E VALORES MOBILIARIOS S.A.'
df_final['VlrEstorno'] = 0

df_final = df_final[['CARTEIRA','DocFundo','Data','NmGestor','NmAdministrador','Entradas','Saídas','VlrEstorno','Qtde Cotas','Cota','Patrimônio']]
df_final.columns = ['NmFundo', 'DocFundo', 'DtPosicao', 'NmGestor', 'NmAdministrador', 'VlrAplicacao', 'VlrResgate','VlrEstorno','QtdCota' ,'VlrCota', 'VlrPatrimonio']

df_final['DocFundo'] = df_final.apply(lambda row: '41.520.452/0001-81' if 'AGROFORTE' in row['DocFundo'] else row['DocFundo'], axis=1)
df_final['DocFundo'] = df_final.apply(lambda row: '51.660.048/0001-67' if 'APG' in row['DocFundo'] else row['DocFundo'], axis=1)
df_final['DocFundo'] = df_final.apply(lambda row: '33.499.726/0001-70' if 'BRISTOL FIDC MULTISSETORIAL' in row['DocFundo'] else row['DocFundo'], axis=1)
df_final['DocFundo'] = df_final.apply(lambda row: '33.499.726/0001-70' if 'BRISTOL FIDC MULTISSETORIAL SENIOR 5' in row['DocFundo'] else row['DocFundo'], axis=1)
df_final['DocFundo'] = df_final.apply(lambda row: '33.499.726/0001-70' if 'BRISTOL FIDC MULTISSETORIAL MEZ CLA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_final['DocFundo'] = df_final.apply(lambda row: '33.499.726/0001-70' if 'BRISTOL FIDC MULTISSETORIAL MEZ CLC' in row['DocFundo'] else row['DocFundo'], axis=1)
df_final['DocFundo'] = df_final.apply(lambda row: '33.499.726/0001-70' if 'BRISTOL FIDC MULTISSETORIAL MEZ HIGH YIELD' in row['DocFundo'] else row['DocFundo'], axis=1)
df_final['DocFundo'] = df_final.apply(lambda row: '33.499.726/0001-70' if 'BRISTOL FIDC MULTISSETORIAL MEZ CLB' in row['DocFundo'] else row['DocFundo'], axis=1)
df_final['DocFundo'] = df_final.apply(lambda row: '33.499.726/0001-70' if 'BRISTOL FIDC MULTISSETORIAL SENIOR 2' in row['DocFundo'] else row['DocFundo'], axis=1)
df_final['DocFundo'] = df_final.apply(lambda row: '33.499.726/0001-70' if 'BRISTOL FIDC MULTISSETORIAL SENIOR 4' in row['DocFundo'] else row['DocFundo'], axis=1)
df_final['DocFundo'] = df_final.apply(lambda row: '33.499.726/0001-70' if 'BRISTOL FIDC MULTISSETORIAL SENIOR 1' in row['DocFundo'] else row['DocFundo'], axis=1)



date_columns = ['DtPosicao']
df_final[date_columns] = df_final[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")

columns_to_convert = ["VlrAplicacao", "VlrResgate", "VlrEstorno","QtdCota", "VlrCota", "VlrPatrimonio"]

def transformar_valor(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    return float(valor_str)

for column in columns_to_convert:
    df_final[column] = df_final[column].apply(transformar_valor)
    
# Converter valores positivos da coluna VlrResgate para negativos
df_final['VlrResgate'] = df_final['VlrResgate'].apply(lambda x: -abs(x) if x > 0 else x)


df_final[["VlrAplicacao","VlrResgate","VlrEstorno","QtdCota","VlrCota","VlrPatrimonio"]].head()

KeyError: 'CARTEIRA'

In [6]:
df_final.loc[df_final['NmFundo'] == 'AGROFORTE FIDC - SUBORDINADA', 'NmFundo'] = 'FIDC AGROFORTE'
df_final.loc[df_final['NmFundo'] == 'AGROFORTE FIDC - MEZANINO 1', 'NmFundo'] = 'FIDC AGROFORTE MZ1'
df_final.loc[df_final['NmFundo'] == 'AGROFORTE FIDC - MEZANINO 2', 'NmFundo'] = 'FIDC AGROFORTE MZ2'
df_final.loc[df_final['NmFundo'] == 'AGROFORTE FIDC - SENIOR 1', 'NmFundo'] = 'FIDC AGROFORTE SR1'
df_final.loc[df_final['NmFundo'] == 'AGROFORTE FIDC - SENIOR 2', 'NmFundo'] = 'FIDC AGROFORTE SR2'
df_final.loc[df_final['NmFundo'] == 'APG FIDC - SUBORDINADA', 'NmFundo'] = 'FIDC APG'
df_final.loc[df_final['NmFundo'] == 'APG FIDC - MEZANINO 1', 'NmFundo'] = 'FIDC APG MZ1'
df_final.loc[df_final['NmFundo'] == 'APG FIDC - MEZANINO 2', 'NmFundo'] = 'FIDC APG MZ2'
df_final.loc[df_final['NmFundo'] == 'APG FIDC - SENIOR 1', 'NmFundo'] = 'FIDC APG SR1'
df_final.loc[df_final['NmFundo'] == 'APG FIDC - SENIOR 2', 'NmFundo'] = 'FIDC APG SR2'
df_final.loc[df_final['NmFundo'] == 'BRISTOL FIDC MULTISSETORIAL', 'NmFundo'] = 'FIDC BRISTOL'
df_final.loc[df_final['NmFundo'] == 'BRISTOL FIDC MULTISSETORIAL SENIOR 5', 'NmFundo'] = 'FIDC BRISTOL SR5'
df_final.loc[df_final['NmFundo'] == 'BRISTOL FIDC MULTISSETORIAL MEZ CLA', 'NmFundo'] = 'FIDC BRISTOL CLA MZ1'
df_final.loc[df_final['NmFundo'] == 'BRISTOL FIDC MULTISSETORIAL MEZ CLC', 'NmFundo'] = 'FIDC BRISTOL CLC MZ2'
df_final.loc[df_final['NmFundo'] == 'BRISTOL FIDC MULTISSETORIAL MEZ CLB', 'NmFundo'] = 'FIDC BRISTOL CLB MZ3'
df_final.loc[df_final['NmFundo'] == 'BRISTOL FIDC MULTISSETORIAL MEZ HIGH YIELD', 'NmFundo'] = 'FIDC BRISTOL HY MZ4'
df_final.loc[df_final['NmFundo'] == 'BRISTOL FIDC MULTISSETORIAL SENIOR 2', 'NmFundo'] = 'FIDC BRISTOL SR2'
df_final.loc[df_final['NmFundo'] == 'BRISTOL FIDC MULTISSETORIAL SENIOR 4', 'NmFundo'] = 'FIDC BRISTOL SR4'
df_final.loc[df_final['NmFundo'] == 'BRISTOL FIDC MULTISSETORIAL SENIOR 1', 'NmFundo'] = 'FIDC BRISTOL SR1'


df_final[['NmFundo', 'DocFundo']].drop_duplicates()

KeyError: 'NmFundo'

In [7]:
df_final

""


In [8]:
print(f"Min: {df_final.DtPosicao.min()} Max: {df_final.DtPosicao.max()}")

AttributeError: 'DataFrame' object has no attribute 'DtPosicao'

In [9]:
import mysql.connector

# Função para inserir um pedaço do DataFrame na tabela MySQL
def insert_dataframe_to_mysql(df, connection, table_name):
    cursor = connection.cursor()
    for i, row in df.iterrows():
        sql = f"INSERT INTO {table_name} ({', '.join(row.index)}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, tuple(row))
    connection.commit()
    cursor.close()

# Conectar ao banco de dados MySQL
connection = mysql.connector.connect(
    host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
    database='DW_CORPORATIVO',
    user='admin',
    password='yb222KraY7PTN0jbH7P3'
)

if connection.is_connected():
    print('Conectado ao MySQL')

    # Dividir o DataFrame em pedaços menores para evitar problemas de memória
    tamanho_do_pedaço = 1000  # Ajuste conforme necessário
    total_registros = len(df_final)

    for i in range(0, total_registros, tamanho_do_pedaço):
        pedaco_df = df_final.iloc[i:i+tamanho_do_pedaço]
        insert_dataframe_to_mysql(pedaco_df, connection, 'Ft_Patrimonio')
        print(f"Registros inseridos: {i + len(pedaco_df)} / {total_registros}")

    print('Concluído')
else:
    print('Falha na conexão ao MySQL')

# Fechar a conexão com o banco de dados
connection.close()

Conectado ao MySQL
Concluído


In [10]:
# Registrar log Patrimônio Singulare 
start_time = time.time()
db_conn = get_db_connection()
qtd_antes = obter_qtd_registros('Ft_Patrimonio', db_conn)

insert_dataframe_to_mysql(df_final, db_conn, 'Ft_Patrimonio')
qtd_depois = obter_qtd_registros('Ft_Patrimonio', db_conn)
qtd_inseridos = qtd_depois - qtd_antes
qtd_ausentes, detalhes_ausentes, fundos_ausentes = verificar_registros_ausentes(db_conn)
duracao_execucao = round(time.time() - start_time, 2)
registrar_log('Patrimônio Singulare', qtd_inseridos, qtd_ausentes, detalhes_ausentes, fundos_ausentes, duracao_execucao, db_conn)
db_conn.close()

______________

# DAYCOVAL


In [11]:
import pandas as pd
import os
arquivos = os.listdir("//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Patrimonio/")
## Rentabilidade Sintética


for i in range(len(arquivos)):
    print(f"[{i}] - {arquivos[i]}")

[0] - R0001 - Rentabilidade Sintetica.csv


In [12]:
x=0
arquivos[x]

'R0001 - Rentabilidade Sintetica.csv'

In [13]:
df_csv = pd.read_csv(f'//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Patrimonio/{arquivos[x]}',delimiter=';',encoding='latin-1', dtype='str')

df_csv = df_csv.iloc[::-1]

colunas = []
for i in range(len(df_csv.columns)):
    x = df_csv.iloc[0][i]
    colunas.append(x) 

### salva priemeira linha que está como coluna
values_primeira_linha = df_csv.columns

df_csv.columns = colunas

### exclui 2 primeiras linhas
df_csv = df_csv.iloc[2:]


# df_csv.iloc[0, :] = values_primeira_linha
df_csv.loc[len(df_csv)] = values_primeira_linha

df_csv['DocFundo'] = df_csv['NomeCrt']
df_csv['NmGestor'] = 'CATALISE INVEST LTDA'
df_csv['VlrEstorno'] = 0

df_csv = df_csv[['NomeCrt','DocFundo','DataRef','NmGestor','Adm','Captacao','Resgate','VlrEstorno','NCotas','ValCota','PL']]
df_csv.columns = ['NmFundo', 'DocFundo', 'DtPosicao', 'NmGestor', 'NmAdministrador', 'VlrAplicacao', 'VlrResgate','VlrEstorno','QtdCota','VlrCota', 'VlrPatrimonio']

df_csv['NmFundo'] = df_csv['NmFundo'].str.strip() # remove espaços

df_csv['DocFundo'] = df_csv.apply(lambda row: '35.689.601/0001-38' if 'ÁGIS - FIDC NP' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '52.997.828/0001-60' if 'ÁGIS 2 FIDC' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '53.073.677/0001-16' if 'ALBAREDO FIDC                           ' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '53.429.996/0001-11' if 'SC FUNDO DE INVESTIMENTO EM DC - LR' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '51.765.082/0001-04' if 'CATALISE FIC FIM CP' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '53.962.745/0001-06' if 'GLOBAL FUTURA FIDC RL' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '43.290.912/0001-58' if 'SDL - FUNDO DE INVESTIMENTO EM DIREITOS' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '53.783.414/0001-09' if 'PRIME AGRO FIDC RL' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '53.861.634/0001-03' if 'GRUPO PRIME AGRO FIC FIM RESPONSABILIDAD' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '30.142.432/0001-06' if 'IPE-FUNDO DE INVESTIMENTO EM DIREITOS NP' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '53.160.371/0001-05' if 'Z INVEST FIDC' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '30.142.432/0001-06' if 'IPE-FUNDO DE INVESTIMENTO EM DIREITOS NP' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '45.425.498/0001-72' if 'K-FINANCE FIDC NP - SUBORDINADA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '53.962.745/0001-06' if 'GLOBAL FUTURA FIDC RL FIDC' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '47.862.139/0001-08' if 'BLUE ROCKET FIDC  - SUBORDINADA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '47.862.139/0001-08' if 'BLUE ROCKET FIDC  - SENIOR' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '45.425.498/0001-72' if 'K-FINANCE FIDC NP - SENIOR' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '52.511.402/0001-54' if 'FIDC PROMETHEUS' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '53.962.432/0001-40' if 'BASÃ- FUNDO DE INVESTIMENTO EM DIREITOS' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '51.774.918/0001-29' if 'AF6 FIDC - NP' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '52.766.910/0001-83' if 'BELL FUNDO DE INVESTIMENTO EM DIREITOS C' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '52.532.399/0001-55' if 'VERGINIA FUNDO DE INVESTIMENTO EM DIREIT' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '54.218.786/0001-47' if 'ABIATAR FIC DE FIM RESPONSABILIDADE LIMI' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '48.984.429/0001-97' if 'VISHNU FUNDO DE INVESTIMENTO EM DIREITOS' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '35.755.861/0001-64' if 'GOLIATH FUNDO DE INVESTIMENTO MULTIMERCA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '35.755.734/0001-65' if 'ARTANIS FUNDO DE INVESTIMENTO MULTIMERCA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '35.755.797/0001-11' if 'ARCTURUS FIM CP IE' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '48.928.987/0001-35' if 'CREDIAL BANK PAN FIDC' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '44.302.374/0001-37' if 'CREDILOG - FUNDO DE INVESTIMENTO EM DIRE' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '54.618.084/0001-50' if 'BONTEMPO FIDC  - RL                     ' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '48.984.429/0001-97' if 'VISHNU FUNDO DE INVESTIMENTO EM DIREITOS' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '54.851.716/0001-21' if 'PINPAG  FIDC - RL' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.122.120/0001-53' if 'PETKOW CAPITAL FIC DE FIM RESPONSABILIDA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.080.073/0001-22' if 'ADCBANK FIC DE FIM RESPONSABILIDADE LIMI' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.106.118/0001-90' if 'DBANK FUNDO DE INVESTIMENTO EM DIREITOS' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.122.381/0001-73' if 'FUTURO CAPITAL FUNDO DE INVESTIMENTO EM' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '42.462.558/0001-39' if 'KÉRDOS FUNDO DE INVESTIMENTO EM DIREITOS' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '42.479.713/0001-20' if 'VELSO - FUNDO DE INVESTIMENTO EM DIREITO' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '48.348.391/0001-66' if 'AGROCETE FUNDO DE INVESTIMENTO EM DIREIT' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '36.544.636/0001-41' if 'FIDC PAY CARGO' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '36.544.636/0001-41' if 'FIDC PAY CARGO - SN' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '36.399.603/0001-55' if 'GREENWOOD FIDC NP' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.106.451/0001-08' if 'CAPITALIZA FUNDO DE INVESTIMENTO EM DIRE' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '49.250.454/0001-00' if 'NINE CAPITAL FIDC' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.080.475/0001-27' if 'CAP6 FIC FIM RESPONSABILIDADE LIMITADA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.080.368/0001-07' if 'CAP4 FIC FIM RESPONSABILIDADE LIMITADA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '54.842.370/0001-03' if 'BEFIC FUNDO DE INVESTIMENTO EM DIREITOS' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '54.842.157/0001-93' if 'FIDC LIBRA CONSIGNADO' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '51.901.421/0001-24' if 'MASTRENN FIDC - RL' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '52.966.476/0001-85' if 'AVANT FINTECH FIC FIM' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '52.997.521/0001-69' if 'UKF FIDC' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '51.655.558/0001-46' if 'CREDILOG II - FUNDO DE INVESTIMENTO EM D' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '51.655.558/0001-46' if 'CREDILOG II - FUNDO DE INVESTIMENTO - SN' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '52.997.828/0001-60' if 'USECORP CATÁLISE FIDC' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '52.997.828/0001-60' if 'USECORP CATÁLISE FIDC Mezanino' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '52.997.828/0001-60' if 'USECORP CATÁLISE FIDC Sênior' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '51.765.082/0001-04' if 'FICFIDC CATALISE' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '51.765.082/0001-04' if 'CATALISE FIC FIDC - RL' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '43.722.793/0001-65' if 'FIDC F2 BANK MEZANINO' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '43.722.793/0001-65' if 'FIDC F2 BANK SUBORDINADA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '47.350.379/0001-23' if 'MF HOLDING GROUP FIC FIM' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '47.424.724/0001-26' if 'FIDC MF GROUP' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.948.310/0001-24' if 'NR11 FUNDO DE INVESTIMENTO EM DIREITOS C' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.670.650/0001-36' if 'LC AZUL FIC DE FIM RESP LIMITADA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.657.044/0001-80' if 'ANIL FUNDO DE INVESTIMENTO EM DIREITOS C' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '51.328.993/0001-66' if 'NINE CAPITAL FIC DE FIM CP' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '54.842.813/0001-58' if 'BELLIN FIC FIM RESPONSABILIDADE LIMTADA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.105.668/0001-95' if 'VITTRA FUNDO DE INVESTIMENTO EM DIREITOS' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.675.974/0001-67' if 'ANVERES FUNDO DE INVESTIMENTO EM DIREITO' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.694.682/0001-71' if 'ANTARES FIC FIM RESPONSABILIDAD' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.106.118/0001-90' if 'DBANK FIDC SÊNIOR' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '56.055.581/0001-13' if 'SMT AGRO HOLDING FIC FIM' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '56.197.290/0001-60' if 'SMT AGRO FUNDO DE INVESTIMENTO EM DIREIT' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.652.863/0001-35' if 'CONDOBEM FUNDO DE INVESTIMENTO EM DIREIT' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '56.823.795/0001-92' if 'RHB CRED FIDC RL' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '52.041.044/0001-63' if 'MALBEC FIDC - RL' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '40.975.325/0001-04' if '3RD - FUNDO DE INVESTIMENTO EM DIREITOS ' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '56.883.223/0001-07' if 'LEX CAPITAL FIDC - RL' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.652.863/0001-35' if 'CONDOBEM FIDC SENIOR' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '57.378.455/0001-62' if 'TERTON FUNDO DE INVEST. EM DIREITOS - RL' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '57.935.598/0001-28' if 'MACANAN FIC FIM RESPONSABILIDADE LIMITAD' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '56.973.948/0001-88' if 'TORONTO  FIDC RESP LIMITADA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '40.141.299/0001-19' if 'ARABAN FIC FIM RESPONSABILIDADE LIMITAD' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '54.851.716/0001-21' if 'PINPAG SN FIDC - RL' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '55.952.408/0001-55' if 'JAQUIM FIC FIM' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '56.973.948/0001-88' if 'TORONTO FIDC RESP LIMITADA SEN' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '57.810.758/0001-02' if 'YELLOW CAPITAL FIDC RL' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '56.973.948/0001-88' if 'TORONTO FIDC RESP LIMITADA MEZ' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '57.941.435/0001-58' if 'EJM FUNDO DE INVESTIMENTO EM DC - RL' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '46.268.912/0001-40' if 'LOGFIN FIDC' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '58.531.844/0001-49' if 'MENEGOTTI FIC FIM' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '58.495.913/0001-06' if 'ZAB LEGACY FIDC RESP LIMITADA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '57.810.758/0001-02' if 'YELLOW FIDC SN' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '58.739.374/0001-03' if 'SAGEL BANK FIDC RESP LTDA' in row['DocFundo'] else row['DocFundo'], axis=1)
df_csv['DocFundo'] = df_csv.apply(lambda row: '59.076.922/0001-25' if 'SAGEL FIC FIM RESP LTDA' in row['DocFundo'] else row['DocFundo'], axis=1)

date_columns = ['DtPosicao']
df_csv[date_columns] = df_csv[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")

columns_to_convert = ["VlrAplicacao", "VlrResgate", "VlrEstorno","QtdCota", "VlrCota", "VlrPatrimonio"]

def transformar_valor(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    return float(valor_str)

for column in columns_to_convert:
    df_csv[column] = df_csv[column].apply(transformar_valor)

df_csv['VlrAplicacao'] = round(df_csv['VlrAplicacao'],2)
df_csv['VlrResgate'] = round(df_csv['VlrResgate'],2)
df_csv['VlrEstorno'] = round(df_csv['VlrEstorno'],2)
df_csv['VlrPatrimonio'] = round(df_csv['VlrPatrimonio'],2)
df_csv['QtdCota'] = round(df_csv['QtdCota'],8)
df_csv['VlrCota'] = round(df_csv['VlrCota'],8)

df_csv['NmFundo'] = df_csv['NmFundo'].str.strip()

df_csv = df_csv.query("VlrPatrimonio > 0")

df_csv.loc[df_csv['NmFundo'] == 'ABX9 FIM CP', 'NmFundo'] = 'FIM ABX9'
df_csv.loc[df_csv['NmFundo'] == 'ADCBANK FIC DE FIM RESPONSABILIDADE LIMI', 'NmFundo'] = 'FICFIM ADCBANK'
df_csv.loc[df_csv['NmFundo'] == 'AF FC FIM CP RESP LT', 'NmFundo'] = 'FICFIM AF'
df_csv.loc[df_csv['NmFundo'] == 'AGROFORTE FIDC - MEZANINO 1', 'NmFundo'] = 'FIDC AGROFORTE MZ1'
df_csv.loc[df_csv['NmFundo'] == 'AGROFORTE FIDC - SENIOR 1', 'NmFundo'] = 'FIDC AGROFORTE SR1'
df_csv.loc[df_csv['NmFundo'] == 'AGROFORTE FIDC - SENIOR 2', 'NmFundo'] = 'FIDC AGROFORTE SR2'
df_csv.loc[df_csv['NmFundo'] == 'ANTARES FIC FIM RESPONSABILIDAD', 'NmFundo'] = 'FICFIM ANTARES'
df_csv.loc[df_csv['NmFundo'] == 'APG FIDC - MEZANINO 1', 'NmFundo'] = 'FIDC APG MZ1'
df_csv.loc[df_csv['NmFundo'] == 'APG FIDC - MEZANINO 2', 'NmFundo'] = 'FIDC APG MZ2'
df_csv.loc[df_csv['NmFundo'] == 'APG FIDC - SENIOR 1', 'NmFundo'] = 'FIDC APG SR1'
df_csv.loc[df_csv['NmFundo'] == 'APG FIDC - SENIOR 2', 'NmFundo'] = 'FIDC APG SR2'
df_csv.loc[df_csv['NmFundo'] == 'ARABAN FC FIM CP', 'NmFundo'] = 'FICFIM ARABAN'
df_csv.loc[df_csv['NmFundo'] == 'ARABAN FIC FIM RESPONSABILIDADE LIMITAD', 'NmFundo'] = 'FICFIM ARABAN'
df_csv.loc[df_csv['NmFundo'] == 'AROEIRA FIC FIM CP', 'NmFundo'] = 'FICFIM AROEIRA'
df_csv.loc[df_csv['NmFundo'] == 'ATICO FC FIM CP', 'NmFundo'] = 'FICFIM ATICO'
df_csv.loc[df_csv['NmFundo'] == 'ATLANTICO FIC FIM', 'NmFundo'] = 'FICFIM ATLANTICO'
df_csv.loc[df_csv['NmFundo'] == 'BAUNKER II FIM CP', 'NmFundo'] = 'FIM BAUNKER II'
df_csv.loc[df_csv['NmFundo'] == 'CAMPOS GERAIS FC FIM', 'NmFundo'] = 'FICFIM CAMPOS GERAIS'
df_csv.loc[df_csv['NmFundo'] == 'CAP4 FIC FIM RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FICFIM CAP4'
df_csv.loc[df_csv['NmFundo'] == 'CAP6 FIC FIM RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FICFIM CAP6'
df_csv.loc[df_csv['NmFundo'] == 'CARGO HOLD FIC FIM', 'NmFundo'] = 'FICFIM CARGO HOLD'
df_csv.loc[df_csv['NmFundo'] == 'CWB CATALISE FC FIM', 'NmFundo'] = 'FICFIM CWB CATALISE'
df_csv.loc[df_csv['NmFundo'] == 'DANIELI FIC FIM CP', 'NmFundo'] = 'FICFIM DANIELI'
df_csv.loc[df_csv['NmFundo'] == 'DANIELI II FC FIM CP', 'NmFundo'] = 'FICFIM DANIELI II'
df_csv.loc[df_csv['NmFundo'] == 'E11EVEN FIC FIM CP', 'NmFundo'] = 'FICFIM E11EVEN'
df_csv.loc[df_csv['NmFundo'] == 'E3 CAPITAL FIM CP IE', 'NmFundo'] = 'FIM E3 CAPITAL'
df_csv.loc[df_csv['NmFundo'] == 'EARNIER MAPS FIM CP', 'NmFundo'] = 'FIM EARNIER'
df_csv.loc[df_csv['NmFundo'] == 'F2 BANK FC FIM CP', 'NmFundo'] = 'FICFIM F2 BANK'
df_csv.loc[df_csv['NmFundo'] == 'FBH FIC FIM CP', 'NmFundo'] = 'FICFIM FBH'
df_csv.loc[df_csv['NmFundo'] == 'FBJ77 FC FIM CP', 'NmFundo'] = 'FICFIM FBJ77'
df_csv.loc[df_csv['NmFundo'] == 'FFBANK FC FIM', 'NmFundo'] = 'FICFIM FFBANK'
df_csv.loc[df_csv['NmFundo'] == 'FFBANK FC FIM CP', 'NmFundo'] = 'FICFIM FFBANK'
df_csv.loc[df_csv['NmFundo'] == 'FIC ABIATAR', 'NmFundo'] = 'FICFIM ABIATAR'
df_csv.loc[df_csv['NmFundo'] == 'FIC ARCTURUS', 'NmFundo'] = 'FIM ARCTURUS'
df_csv.loc[df_csv['NmFundo'] == 'FIC ARTANIS', 'NmFundo'] = 'FIM ARTANIS'
df_csv.loc[df_csv['NmFundo'] == 'FIC GOLIATH', 'NmFundo'] = 'FIM GOLIATH'
df_csv.loc[df_csv['NmFundo'] == 'FIC PRIME AGRO', 'NmFundo'] = 'FICFIM PRIME AGRO'
df_csv.loc[df_csv['NmFundo'] == 'FIC SMT AGRO', 'NmFundo'] = 'FICFIM SMT AGRO'
df_csv.loc[df_csv['NmFundo'] == 'FIDC 3G BANK NP', 'NmFundo'] = 'FIDC 3G BANK'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA MEZ1', 'NmFundo'] = 'FIDC BETTA MZ1'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA MEZ2', 'NmFundo'] = 'FIDC BETTA MZ2'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA MEZ3', 'NmFundo'] = 'FIDC BETTA MZ3'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA MEZ4', 'NmFundo'] = 'FIDC BETTA MZ4'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA SUB', 'NmFundo'] = 'FIDC BETTA'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA SEN', 'NmFundo'] = 'FIDC BETTA SR'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BR8 MEZ', 'NmFundo'] = 'FIDC BR8 MZ'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BR8 SEN', 'NmFundo'] = 'FIDC BR8 SR'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BR8 SUB', 'NmFundo'] = 'FIDC BR8'
df_csv.loc[df_csv['NmFundo'] == 'FIDC CRDLOG 2SR', 'NmFundo'] = 'FIDC CREDILOG 2 SR'
df_csv.loc[df_csv['NmFundo'] == 'FIDC E3 NP', 'NmFundo'] = 'FIDC E3'
df_csv.loc[df_csv['NmFundo'] == 'FIDC FERPAR NP', 'NmFundo'] = 'FIDC FERPAR'
df_csv.loc[df_csv['NmFundo'] == 'FIDC FINTRUST M', 'NmFundo'] = 'FIDC FINTRUST MZ'
df_csv.loc[df_csv['NmFundo'] == 'FIDC PAYCARGO S', 'NmFundo'] = 'FIDC PAY CARGO SR'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA SR 2', 'NmFundo'] = 'FIDC BETTA SR2'
df_csv.loc[df_csv['NmFundo'] == 'FIDC TOPCRED MA', 'NmFundo'] = 'FIDC TOPCRED MZ1'
df_csv.loc[df_csv['NmFundo'] == 'FIDC TOPCRED MB', 'NmFundo'] = 'FIDC TOPCRED MZ2'
df_csv.loc[df_csv['NmFundo'] == 'FIDC TRACTOR MA', 'NmFundo'] = 'FIDC TRACTOR MZ1'
df_csv.loc[df_csv['NmFundo'] == 'FIDC TRACTOR MB', 'NmFundo'] = 'FIDC TRACTOR MZ2'
df_csv.loc[df_csv['NmFundo'] == 'FIGUEIREDO FIC FIM', 'NmFundo'] = 'FICFIM FIGUEIREDO'
df_csv.loc[df_csv['NmFundo'] == 'GANESHA FIC FIM CP', 'NmFundo'] = 'FICFIM GANESHA'
df_csv.loc[df_csv['NmFundo'] == 'GLOBAL FIC FIM CP', 'NmFundo'] = 'FICFIM GLOBAL'
df_csv.loc[df_csv['NmFundo'] == 'GMOT FC FIM', 'NmFundo'] = 'FICFIM GMOT'
df_csv.loc[df_csv['NmFundo'] == 'GMOT FC FIM CP', 'NmFundo'] = 'FICFIM GMOT'
df_csv.loc[df_csv['NmFundo'] == 'GO TOGETHER FIC FIM', 'NmFundo'] = 'FICFIM GO TOGETHER'
df_csv.loc[df_csv['NmFundo'] == 'GPW INV FIC FIM CP', 'NmFundo'] = 'FICFIM GPW'
df_csv.loc[df_csv['NmFundo'] == 'GRUPO CORES FIC FIM', 'NmFundo'] = 'FICFIM GRUPO CORES'
df_csv.loc[df_csv['NmFundo'] == 'ITC FC FIM CP', 'NmFundo'] = 'FICFIM ITC'
df_csv.loc[df_csv['NmFundo'] == 'MAGNO FIC FIM CP', 'NmFundo'] = 'FICFIM MAGNO'
df_csv.loc[df_csv['NmFundo'] == 'MOTOR FID FIC', 'NmFundo'] = 'FICFIM MOTOR FID'
df_csv.loc[df_csv['NmFundo'] == 'MULTIPLE FIC FIM', 'NmFundo'] = 'FICFIM MULTIPLE'
df_csv.loc[df_csv['NmFundo'] == 'PRIME AGRO FIDC', 'NmFundo'] = 'FIDC PRIME AGRO'
df_csv.loc[df_csv['NmFundo'] == 'PRIME AGRO FIDC RL - SEN', 'NmFundo'] = 'FIDC PRIME AGRO SR'
df_csv.loc[df_csv['NmFundo'] == 'FIDC SAMARA SENIOR', 'NmFundo'] = 'FIDC SAMARA SR'
df_csv.loc[df_csv['NmFundo'] == 'FIDC SAMARA', 'NmFundo'] = 'FIDC SAMARA'
df_csv.loc[df_csv['NmFundo'] == 'UBS AMERICANO FID FIM CP', 'NmFundo'] = 'FICFIM UBS AMERICANO'
df_csv.loc[df_csv['NmFundo'] == 'US PARANA FC FIM', 'NmFundo'] = 'FICFIM US PARANA'
df_csv.loc[df_csv['NmFundo'] == 'USECORP CATALISE FIDC', 'NmFundo'] = 'FIDC USECORP CATALISE'
df_csv.loc[df_csv['NmFundo'] == 'USECORP CATALISE FIDC Mezanino', 'NmFundo'] = 'FIDC USECORP CATALISE MZ'
df_csv.loc[df_csv['NmFundo'] == 'USECORP CATALISE FIDC Sênior', 'NmFundo'] = 'FIDC USECORP CATALISE SR'
df_csv.loc[df_csv['NmFundo'] == 'F2 BANK FC FIM CP', 'NmFundo'] = 'FICFIM F2 BANK'
df_csv.loc[df_csv['NmFundo'] == 'FIC ABIATAR', 'NmFundo'] = 'FICFIM ABIATAR'
df_csv.loc[df_csv['NmFundo'] == 'FIC ARCTURUS', 'NmFundo'] = 'FICFIM ARCTURUS'
df_csv.loc[df_csv['NmFundo'] == 'FIC ARTANIS', 'NmFundo'] = 'FICFIM ARTANIS'
df_csv.loc[df_csv['NmFundo'] == 'FIC GOLIATH', 'NmFundo'] = 'FICFIM GOLIATH'
df_csv.loc[df_csv['NmFundo'] == 'FIDC 3G BANK NP', 'NmFundo'] = 'FIDC 3G BANK'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA MEZ1', 'NmFundo'] = 'FIDC BETTA MEZ1'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA MEZ2', 'NmFundo'] = 'FIDC BETTA MEZ2'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA MEZ3', 'NmFundo'] = 'FIDC BETTA MEZ3'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA MEZ4', 'NmFundo'] = 'FIDC BETTA MEZ4'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BR8 MEZ', 'NmFundo'] = 'FIDC BR8 MZ'
df_csv.loc[df_csv['NmFundo'] == 'FIDC CRDLOG 2SR', 'NmFundo'] = 'FIDC CRDLOG 2SR'
df_csv.loc[df_csv['NmFundo'] == 'FIDC E3 NP', 'NmFundo'] = 'FIDC E3'
df_csv.loc[df_csv['NmFundo'] == 'FIDC FERPAR NP', 'NmFundo'] = 'FIDC FERPAR'
df_csv.loc[df_csv['NmFundo'] == 'FIDC FINTRUST M', 'NmFundo'] = 'FIDC FINTRUST'
df_csv.loc[df_csv['NmFundo'] == 'FIDC PAYCARGO S', 'NmFundo'] = 'FIDC PAYCARGO'
df_csv.loc[df_csv['NmFundo'] == 'FIDC TRACTOR MB', 'NmFundo'] = 'FIDC TRACTOR MB'
df_csv.loc[df_csv['NmFundo'] == 'FIDC UNITY NP', 'NmFundo'] = 'FIDC UNITY'
df_csv.loc[df_csv['NmFundo'] == 'FIGUEIREDO FIC FIM', 'NmFundo'] = 'FICFIM FIGUEIREDO'
df_csv.loc[df_csv['NmFundo'] == 'GANESHA FIC FIM CP', 'NmFundo'] = 'FICFIM GANESHA'
df_csv.loc[df_csv['NmFundo'] == 'GLOBAL FIC FIM CP', 'NmFundo'] = 'FICFIM GLOBAL'
df_csv.loc[df_csv['NmFundo'] == 'GMOT FC FIM', 'NmFundo'] = 'FICFIM GMOT'
df_csv.loc[df_csv['NmFundo'] == 'GMOT FC FIM CP', 'NmFundo'] = 'FICFIM GMOT'
df_csv.loc[df_csv['NmFundo'] == 'GO TOGETHER FIC FIM', 'NmFundo'] = 'FICFIM GO TOGETHER'
df_csv.loc[df_csv['NmFundo'] == 'GPW INV FIC FIM CP', 'NmFundo'] = 'FICFIM GPW'
df_csv.loc[df_csv['NmFundo'] == 'GRUPO CORES FIC FIM', 'NmFundo'] = 'FICFIM GRUPO CORES'
df_csv.loc[df_csv['NmFundo'] == 'ITC FC FIM CP', 'NmFundo'] = 'FICFIM ITC'
df_csv.loc[df_csv['NmFundo'] == 'MAGNO FIC FIM CP', 'NmFundo'] = 'FICFIM MAGNO'
df_csv.loc[df_csv['NmFundo'] == 'MOTOR FID FIC', 'NmFundo'] = 'FICFIM MOTOR FID'
df_csv.loc[df_csv['NmFundo'] == 'MULTIPLE FIC FIM', 'NmFundo'] = 'FICFIM MULTIPLE'
df_csv.loc[df_csv['NmFundo'] == 'PRIME AGRO FIDC', 'NmFundo'] = 'FIDC PRIME AGRO'
df_csv.loc[df_csv['NmFundo'] == 'FIDC SAMARA SENIOR', 'NmFundo'] = 'FIDC SAMARA SR'
df_csv.loc[df_csv['NmFundo'] == 'FIDC SAMARA', 'NmFundo'] = 'FIDC SAMARA'
df_csv.loc[df_csv['NmFundo'] == 'UBS AMERICANO FID FIM CP', 'NmFundo'] = 'FICFIM UBS AMERICANO'
df_csv.loc[df_csv['NmFundo'] == 'US PARANA FC FIM', 'NmFundo'] = 'FICFIM US PARANA'
df_csv.loc[df_csv['NmFundo'] == 'USECORP CATALISE FIDC', 'NmFundo'] = 'FIDC USECORP CATALISE'
df_csv.loc[df_csv['NmFundo'] == 'USECORP CATALISE FIDC Mezanino', 'NmFundo'] = 'FIDC USECORP CATALISE MZ'
df_csv.loc[df_csv['NmFundo'] == 'USECORP CATALISE FIDC Sênior', 'NmFundo'] = 'FIDC USECORP CATALISE SR'
df_csv.loc[df_csv['NmFundo'] == 'PETKOW CAPITAL FIC DE FIM RESPONSABILIDA', 'NmFundo'] = 'FICFIM PETKOW CAPITAL'
df_csv.loc[df_csv['NmFundo'] == 'PINPAG SN FIDC - RL', 'NmFundo'] = 'FIDC PINPAG SR'
df_csv.loc[df_csv['NmFundo'] == 'ÁGIS 2 FIDC', 'NmFundo'] = 'FIDC AGIS 2'
df_csv.loc[df_csv['NmFundo'] == 'ÁGIS 2 FIDC Sênior', 'NmFundo'] = 'FIDC AGIS 2 SR'
df_csv.loc[df_csv['NmFundo'] == 'ÁGIS 2 FIDC Mezanino', 'NmFundo'] = 'FIDC AGIS 2 MZ'
df_csv.loc[df_csv['NmFundo'] == 'ALBAREDO FIDC', 'NmFundo'] = 'FIDC ALBAREDO'
df_csv.loc[df_csv['NmFundo'] == 'SC FUNDO DE INVESTIMENTO EM DC - LR', 'NmFundo'] = 'FIDC SC'
df_csv.loc[df_csv['NmFundo'] == 'GLOBAL FUTURA FIDC RL', 'NmFundo'] = 'FIDC GLOBAL FUTURA'
df_csv.loc[df_csv['NmFundo'] == 'SDL - FUNDO DE INVESTIMENTO EM DIREITOS', 'NmFundo'] = 'FIDC SDL'
df_csv.loc[df_csv['NmFundo'] == 'Z INVEST FIDC', 'NmFundo'] = 'FIDC Z INVEST'
df_csv.loc[df_csv['NmFundo'] == 'IPE-FUNDO DE INVESTIMENTO EM DIREITOS NP', 'NmFundo'] = 'FIDC IPE'
df_csv.loc[df_csv['NmFundo'] == 'PRIME AGRO FIDC RL', 'NmFundo'] = 'FIDC PRIME AGRO'
df_csv.loc[df_csv['NmFundo'] == 'GRUPO PRIME AGRO FIC FIM RESPONSABILIDAD', 'NmFundo'] = 'FICFIM PRIME AGRO'
df_csv.loc[df_csv['NmFundo'] == 'K-FINANCE FIDC NP - SUBORDINADA', 'NmFundo'] = 'FIDC K FINANCE'
df_csv.loc[df_csv['NmFundo'] == 'K-FINANCE FIDC NP - SENIOR', 'NmFundo'] = 'FIDC K FINANCE SR'
df_csv.loc[df_csv['NmFundo'] == 'GLOBAL FUTURA FIDC RL FIDC', 'NmFundo'] = 'FIDC GLOBAL FUTURA'
df_csv.loc[df_csv['NmFundo'] == 'BLUE ROCKET FIDC  - SUBORDINADA', 'NmFundo'] = 'FIDC B ROCKET'
df_csv.loc[df_csv['NmFundo'] == 'BLUE ROCKET FIDC  - SENIOR', 'NmFundo'] = 'FIDC B ROCKET SR'
df_csv.loc[df_csv['NmFundo'] == 'BASÃ- FUNDO DE INVESTIMENTO EM DIREITOS', 'NmFundo'] = 'FIDC BASA'
df_csv.loc[df_csv['NmFundo'] == 'AF6 FIDC - NP', 'NmFundo'] = 'FIDC AF6'
df_csv.loc[df_csv['NmFundo'] == 'GLOBAL FUTURA FIDC RL FIDC', 'NmFundo'] = 'FIDC GLOBAL FUTURA'
df_csv.loc[df_csv['NmFundo'] == 'CREDIAL BANK PAN FIDC', 'NmFundo'] = 'FIDC CREDIAL'
df_csv.loc[df_csv['NmFundo'] == 'VERGINIA FUNDO DE INVESTIMENTO EM DIREIT', 'NmFundo'] = 'FIDC VERGINIA'
df_csv.loc[df_csv['NmFundo'] == 'BELL FUNDO DE INVESTIMENTO EM DIREITOS C', 'NmFundo'] = 'FIDC BELL'
df_csv.loc[df_csv['NmFundo'] == 'VISHNU FUNDO DE INVESTIMENTO EM DIREITOS', 'NmFundo'] = 'FIDC VISHNU'
df_csv.loc[df_csv['NmFundo'] == 'ABIATAR FIC DE FIM RESPONSABILIDADE LIMI', 'NmFundo'] = 'FIC ABIATAR'
df_csv.loc[df_csv['NmFundo'] == 'ÁGIS - FIDC NP', 'NmFundo'] = 'FIDC AGIS'
df_csv.loc[df_csv['NmFundo'] == 'FIDC AF6 NP', 'NmFundo'] = 'FIDC AF6'
df_csv.loc[df_csv['NmFundo'] == 'CREDILOG - FUNDO DE INVESTIMENTO EM DIRE', 'NmFundo'] = 'FIDC CREDILOG'
df_csv.loc[df_csv['NmFundo'] == 'BONTEMPO FIDC  - RL', 'NmFundo'] = 'FIDC BONTEMPO'
df_csv.loc[df_csv['NmFundo'] == 'AVANT FINTECH FIC FIM', 'NmFundo'] = 'FICFIM AVANT'
df_csv.loc[df_csv['NmFundo'] == 'KÉRDOS FUNDO DE INVESTIMENTO EM DIREITOS', 'NmFundo'] = 'FIDC KERDOS'
df_csv.loc[df_csv['NmFundo'] == 'IRIS FIC FIM CP', 'NmFundo'] = 'IRIS FIC FIM'
df_csv.loc[df_csv['NmFundo'] == 'DBANK FUNDO DE INVESTIMENTO EM DIREITOS', 'NmFundo'] = 'FIDC DBANK'
df_csv.loc[df_csv['NmFundo'] == 'ARTANIS FUNDO DE INVESTIMENTO MULTIMERCA', 'NmFundo'] = 'FIC ARTANIS'
df_csv.loc[df_csv['NmFundo'] == 'GOLIATH FUNDO DE INVESTIMENTO MULTIMERCA', 'NmFundo'] = 'FIC GOLIATH'
df_csv.loc[df_csv['NmFundo'] == 'ARCTURUS FIM CP IE', 'NmFundo'] = 'FIC ARCTURUS'
df_csv.loc[df_csv['NmFundo'] == 'PINPAG  FIDC - RL', 'NmFundo'] = 'FIDC PINPAG'
df_csv.loc[df_csv['NmFundo'] == 'FUTURO CAPITAL FUNDO DE INVESTIMENTO EM', 'NmFundo'] = 'FIDC FUTURO CAPITAL'
df_csv.loc[df_csv['NmFundo'] == 'VELSO - FUNDO DE INVESTIMENTO EM DIREITO', 'NmFundo'] = 'FIDC VELSO'
df_csv.loc[df_csv['NmFundo'] == 'AGROCETE FUNDO DE INVESTIMENTO EM DIREIT', 'NmFundo'] = 'FIDC AGROCETE'
df_csv.loc[df_csv['NmFundo'] == 'FIDC PAY CARGO - SN', 'NmFundo'] = 'FIDC PAYCARGO S'
df_csv.loc[df_csv['NmFundo'] == 'GREENWOOD FIDC NP', 'NmFundo'] = 'FIDC GREENWOOD'
df_csv.loc[df_csv['NmFundo'] == 'CAPITALIZA FUNDO DE INVESTIMENTO EM DIRE', 'NmFundo'] = 'FIDC CAPITALIZA'
df_csv.loc[df_csv['NmFundo'] == 'NINE CAPITAL FIDC', 'NmFundo'] = 'FIDC NINE'
df_csv.loc[df_csv['NmFundo'] == 'BEFIC FUNDO DE INVESTIMENTO EM DIREITOS', 'NmFundo'] = 'FIDC BEFIC'
df_csv.loc[df_csv['NmFundo'] == 'MASTRENN FIDC - RL', 'NmFundo'] = 'FIDC MASTRENN'
df_csv.loc[df_csv['NmFundo'] == 'FIDC LIBRA CONSIGNADO -SN', 'NmFundo'] = 'FIDC LIBRA CONSIGNADO SR'
df_csv.loc[df_csv['NmFundo'] == 'UKF FIDC', 'NmFundo'] = 'FIDC UKF'
df_csv.loc[df_csv['NmFundo'] == 'CREDILOG II - FUNDO DE INVESTIMENTO - SN', 'NmFundo'] = 'FIDC CRDLOG 2SR'
df_csv.loc[df_csv['NmFundo'] == 'CREDILOG II - FUNDO DE INVESTIMENTO EM D', 'NmFundo'] = 'FIDC CREDILOG 2'
df_csv.loc[df_csv['NmFundo'] == 'USECORP CATÁLISE FIDC', 'NmFundo'] = 'FIDC USECORP CATALISE'
df_csv.loc[df_csv['NmFundo'] == 'USECORP CATÁLISE FIDC Mezanino', 'NmFundo'] = 'FIDC USECORP CATALISE MZ'
df_csv.loc[df_csv['NmFundo'] == 'USECORP CATÁLISE FIDC Sênior', 'NmFundo'] = 'FIDC USECORP CATALISE SR'
df_csv.loc[df_csv['NmFundo'] == 'CATALISE FIDC - RL', 'NmFundo'] = 'FICFIDC CATALISE'
df_csv.loc[df_csv['NmFundo'] == 'FIDC F2 BANK MEZANINO', 'NmFundo'] = 'FIDC F2 BANK MZ'
df_csv.loc[df_csv['NmFundo'] == 'FIDC F2 BANK SUBORDINADA', 'NmFundo'] = 'FIDC F2 BANK'
# df_csv.loc[df_csv['NmFundo'] == 'MF HOLDING GROUP FIC FIM', 'NmFundo'] = 'FICFIM MF HOLDI'
df_csv.loc[df_csv['NmFundo'] == 'NR11 FUNDO DE INVESTIMENTO EM DIREITOS C', 'NmFundo'] = 'FIDC NR11'
df_csv.loc[df_csv['NmFundo'] == 'VITTRA FUNDO DE INVESTIMENTO EM DIREITOS', 'NmFundo'] = 'FIDC VITTRA'
df_csv.loc[df_csv['NmFundo'] == 'ANVERES FUNDO DE INVESTIMENTO EM DIREITO', 'NmFundo'] = 'FIDC ANVERES'
df_csv.loc[df_csv['NmFundo'] == 'ANIL FUNDO DE INVESTIMENTO EM DIREITOS C', 'NmFundo'] = 'FIDC ANIL'
df_csv.loc[df_csv['NmFundo'] == 'DBANK FIDC SÊNIOR', 'NmFundo'] = 'FIDC DBANK SR'
df_csv.loc[df_csv['NmFundo'] == 'RHB CRED FIDC RL', 'NmFundo'] = 'FIDC RHB'
df_csv.loc[df_csv['NmFundo'] == 'SMT AGRO FUNDO DE INVESTIMENTO EM DIREIT', 'NmFundo'] = 'FIDC SMT AGRO'
df_csv.loc[df_csv['NmFundo'] == 'CONDOBEM FUNDO DE INVESTIMENTO EM DIREIT', 'NmFundo'] = 'FIDC CONDOBEM'
df_csv.loc[df_csv['NmFundo'] == '3RD - FUNDO DE INVESTIMENTO EM DIREITOS', 'NmFundo'] = 'FIDC 3RD'
df_csv.loc[df_csv['NmFundo'] == 'MALBEC FIDC - RL', 'NmFundo'] = 'FIDC MALBEC'
df_csv.loc[df_csv['NmFundo'] == 'MACANAN FIC FIM RESPONSABILIDADE LIMITAD', 'NmFundo'] = 'FICFIM MACANAN'
df_csv.loc[df_csv['NmFundo'] == 'TERTON FUNDO DE INVEST. EM DIREITOS - RL', 'NmFundo'] = 'FIDC TERTON'
df_csv.loc[df_csv['NmFundo'] == 'TORONTO  FIDC RESP LIMITADA', 'NmFundo'] = 'FIDC TORONTO'
df_csv.loc[df_csv['NmFundo'] == 'LEX CAPITAL FIDC - RL', 'NmFundo'] = 'FIDC LEX'
df_csv.loc[df_csv['NmFundo'] == 'SMT AGRO HOLDING FIC FIM', 'NmFundo'] = 'FICFIM SMT AGRO'
df_csv.loc[df_csv['NmFundo'] == 'LC AZUL FIC DE FIM RESP LIMITADA', 'NmFundo'] = 'FICFIM LC AZUL'
df_csv.loc[df_csv['NmFundo'] == 'MF HOLDING GROUP FIC FIM', 'NmFundo'] = 'FICFIM MF HOLDING'
df_csv.loc[df_csv['NmFundo'] == 'NINE CAPITAL FIC DE FIM CP', 'NmFundo'] = 'FICFIM NINE CAPITAL'
df_csv.loc[df_csv['NmFundo'] == 'CATALISE FIC FIDC - RL', 'NmFundo'] = 'FICFIDC CATALISE'
df_csv.loc[df_csv['NmFundo'] == 'CONDOBEM FIDC SENIOR', 'NmFundo'] = 'FIDC CONDOBEM SR'
df_csv.loc[df_csv['NmFundo'] == 'BELLIN FIC FIM RESPONSABILIDADE LIMTADA', 'NmFundo'] = 'FICFIM BELLIN'
df_csv.loc[df_csv['NmFundo'] == 'ABX9 FIM CP', 'NmFundo'] = 'FIM ABX9'
df_csv.loc[df_csv['NmFundo'] == 'AF FC FIM CP RESP LT', 'NmFundo'] = 'FICFIM AF'
df_csv.loc[df_csv['NmFundo'] == 'APG FIDC - MEZANINO 1', 'NmFundo'] = 'FIDC APG MZ1'
df_csv.loc[df_csv['NmFundo'] == 'APG FIDC - MEZANINO 2', 'NmFundo'] = 'FIDC APG MZ2'
df_csv.loc[df_csv['NmFundo'] == 'APG FIDC - SENIOR 1', 'NmFundo'] = 'FIDC APG SR1'
df_csv.loc[df_csv['NmFundo'] == 'APG FIDC - SENIOR 2', 'NmFundo'] = 'FIDC APG SR2'
df_csv.loc[df_csv['NmFundo'] == 'AROEIRA FIC FIM CP', 'NmFundo'] = 'FICFIM AROEIRA'
df_csv.loc[df_csv['NmFundo'] == 'ATICO FC FIM CP', 'NmFundo'] = 'FICFIM ATICO'
df_csv.loc[df_csv['NmFundo'] == 'ATLANTICO FIC FIM', 'NmFundo'] = 'FICFIM ATLANTICO'
df_csv.loc[df_csv['NmFundo'] == 'BAUNKER II FIM CP', 'NmFundo'] = 'FIM BAUNKER II'
df_csv.loc[df_csv['NmFundo'] == 'CAMPOS GERAIS FC FIM', 'NmFundo'] = 'FICFIM CAMPOS GERAIS'
df_csv.loc[df_csv['NmFundo'] == 'CARGO HOLD FIC FIM', 'NmFundo'] = 'FICFIM CARGO HOLD'
df_csv.loc[df_csv['NmFundo'] == 'CWB CATALISE FC FIM', 'NmFundo'] = 'FICFIM CWB CATALISE'
df_csv.loc[df_csv['NmFundo'] == 'DANIELI FIC FIM CP', 'NmFundo'] = 'FICFIM DANIELI'
df_csv.loc[df_csv['NmFundo'] == 'DANIELI II FC FIM CP', 'NmFundo'] = 'FICFIM DANIELI II'
df_csv.loc[df_csv['NmFundo'] == 'E11EVEN FIC FIM CP', 'NmFundo'] = 'FICFIM E11EVEN'
df_csv.loc[df_csv['NmFundo'] == 'EARNIER MAPS FIM CP', 'NmFundo'] = 'FIM EARNIER'
df_csv.loc[df_csv['NmFundo'] == 'F2 BANK FC FIM CP', 'NmFundo'] = 'FICFI00M F2 BANK'
df_csv.loc[df_csv['NmFundo'] == 'FBH FIC FIM CP', 'NmFundo'] = 'FICFIM FBH'
df_csv.loc[df_csv['NmFundo'] == 'FBJ77 FC FIM CP', 'NmFundo'] = 'FICFIM FBJ77'
df_csv.loc[df_csv['NmFundo'] == 'FFBANK FC FIM', 'NmFundo'] = 'FICFIM FFBANK'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA MEZ4', 'NmFundo'] = 'FIDC BETTA MZ4'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA SR 2', 'NmFundo'] = 'FIDC BETTA SR2'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BETTA SUB', 'NmFundo'] = 'FIDC BETTA'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BR8 MEZ', 'NmFundo'] = 'FIDC BR8 MZ'
df_csv.loc[df_csv['NmFundo'] == 'FIDC BR8 SUB', 'NmFundo'] = 'FIDC BR8'
df_csv.loc[df_csv['NmFundo'] == 'FIDC CRDLOG 2SR', 'NmFundo'] = 'FIDC CREDILOG 2 SR'
df_csv.loc[df_csv['NmFundo'] == 'FIDC FINANZE SB', 'NmFundo'] = 'FIDC FINANZE'
df_csv.loc[df_csv['NmFundo'] == 'FIDC FINTRUST M', 'NmFundo'] = 'FIDC FINTRUST MZ'
df_csv.loc[df_csv['NmFundo'] == 'FIDC NINE', 'NmFundo'] = 'FIDC NINE CAPITAL'
df_csv.loc[df_csv['NmFundo'] == 'FIDC PAYCARGO S', 'NmFundo'] = 'FIDC PAY CARGO SR'
df_csv.loc[df_csv['NmFundo'] == 'FIDC TOPCRED MA', 'NmFundo'] = 'FIDC TOPCRED MZ1'
df_csv.loc[df_csv['NmFundo'] == 'FIDC TOPCRED MB', 'NmFundo'] = 'FIDC TOPCRED MZ2'
df_csv.loc[df_csv['NmFundo'] == 'FIDC TRACTOR MA', 'NmFundo'] = 'FIDC TRACTOR MZ1'
df_csv.loc[df_csv['NmFundo'] == 'FIDC UNITY NP', 'NmFundo'] = 'FIDC UNITY'
df_csv.loc[df_csv['NmFundo'] == 'FIGUEIREDO FIC FIM', 'NmFundo'] = 'FICFIM FIGUEIREDO'
df_csv.loc[df_csv['NmFundo'] == 'GANESHA FIC FIM CP', 'NmFundo'] = 'FICFIM GANESHA'
df_csv.loc[df_csv['NmFundo'] == 'GLOBAL FIC FIM CP', 'NmFundo'] = 'FICFIM GLOBAL'
df_csv.loc[df_csv['NmFundo'] == 'GMOT FC FIM', 'NmFundo'] = 'FICFIM GMOT'
df_csv.loc[df_csv['NmFundo'] == 'GO TOGETHER FIC FIM', 'NmFundo'] = 'FICFIM GO TOGETHER'
df_csv.loc[df_csv['NmFundo'] == 'GPW INV FIC FIM CP', 'NmFundo'] = 'FICFIM GPW'
df_csv.loc[df_csv['NmFundo'] == 'GRUPO CORES FIC FIM', 'NmFundo'] = 'FICFIM GRUPO CORES'
df_csv.loc[df_csv['NmFundo'] == 'JB FIM CP', 'NmFundo'] = 'FIM JB'
df_csv.loc[df_csv['NmFundo'] == 'LIBRA FC FIM CP', 'NmFundo'] = 'FICFIM LIBRA'
df_csv.loc[df_csv['NmFundo'] == 'LION FC FIDC', 'NmFundo'] = 'FICFIDC LION'
df_csv.loc[df_csv['NmFundo'] == 'MASTRENN FIC FIM CP', 'NmFundo'] = 'FICFIM MASTRENN'
df_csv.loc[df_csv['NmFundo'] == 'NOVA GLOBAL FIC FIM', 'NmFundo'] = 'FICFIM NOVA GLOBAL'
df_csv.loc[df_csv['NmFundo'] == 'OIKOS FC FIM CP', 'NmFundo'] = 'FICFIM OIKOS'
df_csv.loc[df_csv['NmFundo'] == 'PAN FC FIM CP', 'NmFundo'] = 'FICFIM PAN'
df_csv.loc[df_csv['NmFundo'] == 'PROSPER FIM CP', 'NmFundo'] = 'FIM PROSPER'
df_csv.loc[df_csv['NmFundo'] == 'RHB INVEST FIC FIM', 'NmFundo'] = 'FICFIM RHB'
df_csv.loc[df_csv['NmFundo'] == 'RINCAO FIC FIM', 'NmFundo'] = 'FICFIM RINCAO'
df_csv.loc[df_csv['NmFundo'] == 'SCI SAO CR FC FIM CP', 'NmFundo'] = 'FICFIM SCI'
df_csv.loc[df_csv['NmFundo'] == 'SECULO FIC FIM CP', 'NmFundo'] = 'FICFIM SECULO'
df_csv.loc[df_csv['NmFundo'] == 'UKF FIC FIM', 'NmFundo'] = 'FICFIM UKF'
df_csv.loc[df_csv['NmFundo'] == 'VEDELAGO FIC FIM', 'NmFundo'] = 'FICFIM VEDALAGO'
df_csv.loc[df_csv['NmFundo'] == 'Z CORP FIC FIM', 'NmFundo'] = 'FICFIM Z CORP'
df_csv.loc[df_csv['NmFundo'] == 'JAQUIM FIC FIM', 'NmFundo'] = 'FICFIM JAQUIM'
df_csv.loc[df_csv['NmFundo'] == 'TORONTO FIDC RESP LIMITADA SEN', 'NmFundo'] = 'FIDC TORONTO SR'
df_csv.loc[df_csv['NmFundo'] == 'YELLOW CAPITAL FIDC RL', 'NmFundo'] = 'FIDC YELLOW'
df_csv.loc[df_csv['NmFundo'] == 'TORONTO FIDC RESP LIMITADA MEZ', 'NmFundo'] = 'FIDC TORONTO MZ'
df_csv.loc[df_csv['NmFundo'] == 'EJM FUNDO DE INVESTIMENTO EM DC - RL', 'NmFundo'] = 'FIDC EJM'
df_csv.loc[df_csv['NmFundo'] == 'LOGFIN FIDC', 'NmFundo'] = 'FIDC LOGFIN'
df_csv.loc[df_csv['NmFundo'] == 'ZAB LEGACY FIDC RESP LIMITADA', 'NmFundo'] = 'FIDC ZAB LEGACY'
df_csv.loc[df_csv['NmFundo'] == 'MENEGOTTI FIC FIM', 'NmFundo'] = 'FICFIM MENEGOTTI'
df_csv.loc[df_csv['NmFundo'] == 'YELLOW FIDC SN', 'NmFundo'] = 'FIDC YELLOW'
df_csv.loc[df_csv['NmFundo'] == 'SAGEL BANK FIDC RESP LTDA', 'NmFundo'] = 'FIDC SAGEL BANK'
df_csv.loc[df_csv['NmFundo'] == 'SAGEL FIC FIM RESP LTDA', 'NmFundo'] = 'FICFIM SAGEL ASSESSORIA'



df_csv['VlrAplicacao'].fillna(0, inplace=True)
df_csv['VlrResgate'].fillna(0, inplace=True)


#df_csv[['NmFundo','DocFundo']].drop_duplicates().sort_values(ascending=True)

# Converter valores positivos da coluna VlrResgate para negativos
df_csv['VlrResgate'] = df_csv['VlrResgate'].apply(lambda x: -abs(x) if x > 0 else x)

In [14]:
df_csv[['NmFundo','DocFundo','DtPosicao']].drop_duplicates().head(50)

,NmFundo,DocFundo,DtPosicao
92,FIDC F2 BANK MZ,43.722.793/0001-65,2025-03-20
91,FIDC TORONTO MZ,56.973.948/0001-88,2025-03-20
90,FIDC CREDILOG 2 SR,51.655.558/0001-46,2025-03-20
89,FIDC USECORP CATALISE MZ,52.997.828/0001-60,2025-03-20
88,FIDC TORONTO SR,56.973.948/0001-88,2025-03-20
86,FIDC DBANK SR,55.106.118/0001-90,2025-03-20
85,FIDC LIBRA CONSIGNADO SR,54.842.157/0001-93,2025-03-20
84,FIDC PINPAG SR,54.851.716/0001-21,2025-03-20
83,FIDC PAY CARGO SR,36.544.636/0001-41,2025-03-20
82,FIDC B ROCKET SR,47.862.139/0001-08,2025-03-20


In [15]:
print(df_csv[['NmFundo','DocFundo']].drop_duplicates().head(200).to_string())

                     NmFundo            DocFundo
92           FIDC F2 BANK MZ  43.722.793/0001-65
91           FIDC TORONTO MZ  56.973.948/0001-88
90        FIDC CREDILOG 2 SR  51.655.558/0001-46
89  FIDC USECORP CATALISE MZ  52.997.828/0001-60
88           FIDC TORONTO SR  56.973.948/0001-88
86             FIDC DBANK SR  55.106.118/0001-90
85  FIDC LIBRA CONSIGNADO SR  54.842.157/0001-93
84            FIDC PINPAG SR  54.851.716/0001-21
83         FIDC PAY CARGO SR  36.544.636/0001-41
82          FIDC B ROCKET SR  47.862.139/0001-08
80        FIDC PRIME AGRO SR  53.783.414/0001-09
79  FIDC USECORP CATALISE SR  52.997.828/0001-60
77               FIDC YELLOW  57.810.758/0001-02
76   FICFIM SAGEL ASSESSORIA  59.076.922/0001-25
74           FIDC SAGEL BANK  58.739.374/0001-03
73           FIDC ZAB LEGACY  58.495.913/0001-06
72          FICFIM MENEGOTTI  58.531.844/0001-49
71             FICFIM ARABAN  40.141.299/0001-19
70            FICFIM MACANAN  57.935.598/0001-28
69                  

In [16]:
df_final = df_csv


### insere no banco 
import mysql.connector

# Função para inserir um pedaço do DataFrame na tabela MySQL
def insert_dataframe_to_mysql(df, connection, table_name):
    cursor = connection.cursor()
    for i, row in df.iterrows():
        sql = f"INSERT INTO {table_name} ({', '.join(row.index)}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, tuple(row))
    connection.commit()
    cursor.close()

# Conectar ao banco de dados MySQL
connection = mysql.connector.connect(
    host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
    database='DW_CORPORATIVO',
    user='admin',
    password='yb222KraY7PTN0jbH7P3'
)

if connection.is_connected():
    print('Conectado ao MySQL')

    # Dividir o DataFrame em pedaços menores para evitar problemas de memória
    tamanho_do_pedaço = 1000  # Ajuste conforme necessário
    total_registros = len(df_final)

    for i in range(0, total_registros, tamanho_do_pedaço):
        pedaco_df = df_final.iloc[i:i+tamanho_do_pedaço]
        insert_dataframe_to_mysql(pedaco_df, connection, 'Ft_Patrimonio')
        print(f"Registros inseridos: {i + len(pedaco_df)} / {total_registros}")

    print('Concluído')
else:
    print('Falha na conexão ao MySQL')

# Fechar a conexão com o banco de dados
connection.close()

Conectado ao MySQL
Registros inseridos: 85 / 85
Concluído


In [17]:
# Registrar log Patrimônio Daycoval
start_time = time.time()
db_conn = get_db_connection()
qtd_antes = obter_qtd_registros('Ft_Patrimonio', db_conn)

insert_dataframe_to_mysql(df_csv, db_conn, 'Ft_Patrimonio')
qtd_depois = obter_qtd_registros('Ft_Patrimonio', db_conn)
qtd_inseridos = qtd_depois - qtd_antes
qtd_ausentes, detalhes_ausentes, fundos_ausentes = verificar_registros_ausentes(db_conn)
duracao_execucao = round(time.time() - start_time, 2)
registrar_log('Patrimônio Daycoval', qtd_inseridos, qtd_ausentes, detalhes_ausentes, fundos_ausentes, duracao_execucao, db_conn)
db_conn.close()

# BTG \ FICs

CotasRentabilidades

In [ ]:
# x=1
import pandas as pd
import os
arquivos = os.listdir("//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Patrimonio/")
# CotasRentabilidades_20240502175628

for i in range(len(arquivos)):
    print(f"[{i}] - {arquivos[i]}")

In [ ]:
# for i in range(len(arquivos)):
for i in range(1):
    df_csv = pd.read_csv(f'//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Patrimonio/{arquivos[1]}',delimiter=',',encoding='latin-1', dtype='str')

    df_csv = df_csv[['Nome da classe', 'CNPJ da classe', 'Data','Aquisição (R$)','Resgate (R$)','Quantidade de cotas','Cota líquida (R$)', 'Patrimônio líquido (R$)']]
    df_csv.columns = ['NmFundo', 'DocFundo', 'DtPosicao','VlrAplicacao', 'VlrResgate','QtdCota','VlrCota', 'VlrPatrimonio']

    date_columns = ['DtPosicao']
    df_csv[date_columns] = df_csv[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")

    columns_to_convert = ["VlrAplicacao","VlrResgate","QtdCota","VlrCota", "VlrPatrimonio"]

    def transformar_valor(valor):
        if pd.isna(valor):
            return valor
        valor_str = str(valor)
        valor_str = valor_str.replace(',', '.')
        return float(valor_str)

    for column in columns_to_convert:
        df_csv[column] = df_csv[column].apply(transformar_valor)
        
    df_csv['NmGestor']='CATALISE INVEST LTDA'
    
    df_csv['NmAdministrador']='BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM'
        
    df_final = df_csv
    df_final = df_final.query("VlrPatrimonio > 0")
    df_final['NmFundo'] = df_final['NmFundo'].str.strip() # remove espaços

    # Converter valores positivos da coluna VlrResgate para negativos
    df_final['VlrResgate'] = df_final['VlrResgate'].apply(lambda x: -abs(x) if x > 0 else x)


        #### insere no banco 
    import mysql.connector

    # Função para inserir um pedaço do DataFrame na tabela MySQL
    def insert_dataframe_to_mysql(df, connection, table_name):
        cursor = connection.cursor()
        for i, row in df.iterrows():
            sql = f"INSERT INTO {table_name} ({', '.join(row.index)}) VALUES ({', '.join(['%s'] * len(row))})"
            cursor.execute(sql, tuple(row))
        connection.commit()
        cursor.close()

    # Conectar ao banco de dados MySQL
    connection = mysql.connector.connect(
        host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
        database='DW_CORPORATIVO',
        user='admin',
        password='yb222KraY7PTN0jbH7P3'
    )

    if connection.is_connected():
        print('Conectado ao MySQL')

        # Dividir o DataFrame em pedaços menores para evitar problemas de memória
        tamanho_do_pedaço = 1000  # Ajuste conforme necessário
        total_registros = len(df_final)

        for i in range(0, total_registros, tamanho_do_pedaço):
            pedaco_df = df_final.iloc[i:i+tamanho_do_pedaço]
            insert_dataframe_to_mysql(pedaco_df, connection, 'Ft_Patrimonio')
            print(f"Registros inseridos: {i + len(pedaco_df)} / {total_registros}")

        print('Concluído')
    else:
        print('Falha na conexão ao MySQL')

In [ ]:
# Registrar log Patrimônio BTG
start_time = time.time()
db_conn = get_db_connection()
qtd_antes = obter_qtd_registros('Ft_Patrimonio', db_conn)

insert_dataframe_to_mysql(df_csv, db_conn, 'Ft_Patrimonio')
qtd_depois = obter_qtd_registros('Ft_Patrimonio', db_conn)
qtd_inseridos = qtd_depois - qtd_antes
qtd_ausentes, detalhes_ausentes, fundos_ausentes = verificar_registros_ausentes(db_conn)
duracao_execucao = round(time.time() - start_time, 2)
registrar_log('Patrimônio BTG', qtd_inseridos, qtd_ausentes, detalhes_ausentes, fundos_ausentes, duracao_execucao, db_conn)
db_conn.close()

print("ETL finalizado com sucesso! Logs registrados.")